<h1 align='center'> Basketball Data Collection </h1>

This notebook contains the associated work necessary to collect the data that composes the [***Kaggle Basketball Dataset*** (wyattowalsh/basketball)](https://www.kaggle.com/wyattowalsh/basketball) and serves as the foundation for the [basketball related projects](https://github.com/wyattowalsh/sports-analytics/tree/main/basketball) within my [sports analytics GitHub repository](https://github.com/wyattowalsh/sports-analytics).

One of the goals for the data collection component of this project is to produce a `robust`, *organized* dataset that can grow to as **large of a scale** as possible. You can find an explanation of my solution for storing the files related to the [***Basketball Dataset***](https://www.kaggle.com/wyattowalsh/basketball) below.

<img src="https://unsplash.com/photos/Kv-gAzpUSRg/download?force=true">

***Kaggle*** offers many formats of which one can save files to a dataset, which include: `CSV`, `JSON`, `SQLite`, and `Archives`, among others. The platform essentially acts similarly to industrial cloud solutions like *Google Cloud Platform's* (**GCP**) ***Cloud Storage*** or *Amazon Web Service's* (**AWS**) ***S3*** albeit with a **100GB** storage capacity. ***Kaggle*** datasets as well as these industrial solutions can be considered as broad object/file storage and in certain data engineering paradigms can serve as data lakes. 

It seems that many state-of-the-art (SOTA) data storage solutions pivot around an organizational-wide data lake (of which itself allows for general object storage) that has multiple inputs (*"tributaries"*) both streaming into and routinely added to the overall lake. One benefit of this paradigm is that the lake facilitates the storage of both structured (tabular) and unstructured (image, video, audio, text, etc) data. This can prove useful because, as time progresses, new techniques for extracting useful information from unstructured data can be utilized. Thus it also seems like a good idea to hold onto all extracted data, if possible. 

***Kaggle*** datasets can serve as data lakes through the archival process or simply storing data files in their raw file format. This certainly serves as a strong foundation for building a &#8212; one day in the future &#8212; <b><i>"big data"</i></b> collection. 

However, there is further work that can be done in configuring ***Kaggle*** datasets to enable additional platform functionality as well as improved storage efficiency. data storage solution of data lakes. Similarly, to how storing files in the `.avro`  Further, data lakes can be refined over time to enable extraction and analysis across all ingested data. One feature of data lakes that can be utilized is storing 




To facilitate the growth of this &#8212; one day in the future &#8212; "big data" collection,  this, the dataset is stored within a ***SQLite*** database ( [`basketball.sqlite`] ). Of the different options available to store data on ***Kaggle*** 

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prepare-Development-Environment" data-toc-modified-id="Prepare-Development-Environment-1">Prepare Development Environment</a></span><ul class="toc-item"><li><span><a href="#Clone-GitHub-Repository-(if-necessary)" data-toc-modified-id="Clone-GitHub-Repository-(if-necessary)-1.1">Clone GitHub Repository (if necessary)</a></span></li><li><span><a href="#Install-Conda-package-manager-(if-necessary)" data-toc-modified-id="Install-Conda-package-manager-(if-necessary)-1.2">Install Conda package manager (if necessary)</a></span></li><li><span><a href="#Install-dependencies" data-toc-modified-id="Install-dependencies-1.3">Install dependencies</a></span></li></ul></li><li><span><a href="#Import-Dependencies,-Initialize-Kaggle,-Initialize-Dask" data-toc-modified-id="Import-Dependencies,-Initialize-Kaggle,-Initialize-Dask-2">Import Dependencies, Initialize Kaggle, Initialize Dask</a></span></li></ul></div>

## Prepare Development Environment

- ### Clone GitHub Repository (if necessary)
- ### Install Conda package manager (if necessary)
- ### Install dependencies

In [18]:
%%bash

# rm -r sample_data 
# git clone https://github.com/wyattowalsh/sports-analytics.git

# # # install conda
# # bash sports-analytics/project_resources/bash_scripts/install_conda_in_colab.sh 

# # install dependencies
pip install -r ../../dependencies/basketball/data_collection.txt 

  Using cached nba_api-1.1.9-py3-none-any.whl (242 kB)
  Using cached python_slugify-4.0.1-py2.py3-none-any.whl
  Using cached text_unidecode-1.3-py2.py3-none-any.whl (78 kB)
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=d16e095bb760bafab595115f2389c43aa5ecbf0b5258234f52d746705733e6e6
  Stored in directory: /Users/wyattowalsh/Library/Caches/pip/wheels/ac/b2/c3/fa4706d469b5879105991d1c8be9a3c2ef329ba9fe2ce5085e
Successfully built kaggle


## Import Dependencies, Initialize Kaggle, Initialize Dask 

In [1]:
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats
import pandas as pd 
import numpy as np
import os
import sqlite3 as sql
import matplotlib.pyplot as plt
import seaborn
import time
from requests.packages.urllib3.exceptions import ProxyError
import urllib.error
import urllib.request
import dask
from dask.distributed import Client, progress, LocalCluster

# !mkdir -p ~/.kaggle/ && mv < your kaggle json path > ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
import kaggle

In [31]:
conn = sql.connect('../data/basketball.sqlite')

In [4]:
df_players = pd.DataFrame(players.get_players()).astype({'id': 'str'})
df_players

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
4496,1627790,Ante Zizic,Ante,Zizic,True
4497,78647,Jim Zoet,Jim,Zoet,False
4498,78648,Bill Zopf,Bill,Zopf,False
4499,1627826,Ivica Zubac,Ivica,Zubac,True


In [25]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4501 entries, 0 to 4500
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          4501 non-null   object
 1   full_name   4501 non-null   object
 2   first_name  4501 non-null   object
 3   last_name   4501 non-null   object
 4   is_active   4501 non-null   bool  
dtypes: bool(1), object(4)
memory usage: 145.2+ KB


In [26]:
df_players.to_sql('Player', conn)

In [4]:
df_teams = pd.DataFrame(teams.get_teams()).astype({'id': 'str'})
df_teams['year_founded'] =  pd.to_datetime(df_teams['year_founded'], format='%Y').dt.year
df_teams

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
6,1610612743,Denver Nuggets,DEN,Nuggets,Denver,Colorado,1976
7,1610612744,Golden State Warriors,GSW,Warriors,Golden State,California,1946
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
9,1610612746,Los Angeles Clippers,LAC,Clippers,Los Angeles,California,1970


In [28]:
df_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            30 non-null     object
 1   full_name     30 non-null     object
 2   abbreviation  30 non-null     object
 3   nickname      30 non-null     object
 4   city          30 non-null     object
 5   state         30 non-null     object
 6   year_founded  30 non-null     int64 
dtypes: int64(1), object(6)
memory usage: 1.8+ KB


In [29]:
df_teams.to_sql('Team', conn)

In [10]:
def get_proxies():
    !wget -O http_proxies.txt "https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=10000&country=all&ssl=all&anonymity=all&simplified=true"

    with open('http_proxies.txt', 'r') as file:
        proxies = file.read().split('\n')
    print("Original number of proxies: ", len(proxies))

    def check_proxies(proxy):
        try:
            urllib.request.urlopen("http://" + proxy, timeout = 10)
        except:
            return proxy

    dead_proxies = []
    for proxy in proxies:
        dead_proxy = dask.delayed(check_proxies)(proxy)
        dead_proxies.append(dead_proxy)

    dead_proxies = dask.persist(*dead_proxies)
    dead_proxies = list(filter(None, dask.compute(*dead_proxies))) 

    [proxies.remove(proxy) for proxy in dead_proxies]
    if "" in proxies:
        proxies.remove("")
    print("Number of proxies alive: ", len(proxies))
    return proxies

In [11]:
player_ids = df_players['id'].values
cluster = LocalCluster(n_workers=8)
c = Client(cluster)
proxies = get_proxies()
c.shutdown()
proxies

--2021-03-18 15:48:18--  https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=10000&country=all&ssl=all&anonymity=all&simplified=true
Resolving api.proxyscrape.com (api.proxyscrape.com)... 151.139.128.11
Connecting to api.proxyscrape.com (api.proxyscrape.com)|151.139.128.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘http_proxies.txt’

http_proxies.txt        [ <=>                ]  24.70K  --.-KB/s    in 0.01s   

2021-03-18 15:48:19 (2.18 MB/s) - ‘http_proxies.txt’ saved [25295]

Original number of proxies:  1248
Number of proxies alive:  68


['154.16.202.22:3128',
 '62.171.138.89:80',
 '191.96.42.80:3128',
 '109.193.195.11:8080',
 '181.15.156.170:80',
 '116.117.134.134:8081',
 '102.129.249.120:8080',
 '119.93.129.34:80',
 '191.96.71.118:3128',
 '46.237.255.4:8080',
 '209.97.150.167:8080',
 '198.199.86.11:8080',
 '88.198.50.103:8080',
 '78.42.42.42:8080',
 '134.3.255.9:8080',
 '116.117.134.134:9999',
 '116.117.134.134:82',
 '37.49.127.236:8080',
 '82.212.62.27:8080',
 '82.212.62.24:8080',
 '138.68.60.8:8080',
 '116.117.134.134:81',
 '5.252.161.48:3128',
 '78.42.42.37:8080',
 '176.9.75.42:3128',
 '5.189.133.231:80',
 '198.199.86.11:3128',
 '116.117.134.134:80',
 '161.35.70.249:3128',
 '154.16.63.16:3128',
 '159.203.61.169:3128',
 '217.8.51.200:3128',
 '176.9.119.170:3128',
 '139.59.1.14:3128',
 '91.89.89.8:8080',
 '46.4.96.137:3128',
 '167.71.5.83:8080',
 '209.97.150.167:3128',
 '103.150.61.197:8080',
 '82.212.62.26:8080',
 '173.212.202.65:80',
 '109.193.195.13:8080',
 '121.206.45.159:4216',
 '220.174.236.211:8091',
 '85.216

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [13]:
def get_quick_proxies():
    !wget -O http_proxies.txt "https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=3500&country=all&ssl=all&anonymity=all&simplified=true"

    with open('http_proxies.txt', 'r') as file:
        proxies = file.read().split('\n')
    print("Original number of proxies: ", len(proxies))

    def check_proxies(proxy):
        try:
            urllib.request.urlopen("http://" + proxy, timeout = 2.5)
        except IOError:
            return proxy

    dead_proxies = []
    for proxy in proxies:
        dead_proxy = dask.delayed(check_proxies)(proxy)
        dead_proxies.append(dead_proxy)

    dead_proxies = dask.compute(*dask.persist(*dead_proxies))

    [proxies.remove(proxy) for proxy in dead_proxies]
    if "" in proxies:
        proxies.remove("")
    print("Number of proxies alive: ", len(proxies))
    return proxies

In [14]:
cluster = LocalCluster(n_workers=8)
c = Client(cluster)

def get_common_player_info(player_id, proxies):
    custom_headers = {
    'Host': 'stats.nba.com',
    'Connection': 'keep-alive',
    'Cache-Control': 'max-age=0',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    }
    dfs = []
    while len(dfs) < 2: 
        for proxy in proxies:
            try:
                res = commonplayerinfo.CommonPlayerInfo(player_id=player_id, proxy="http://" + proxy, timeout=100)
                dfs = res.get_data_frames()
                df = pd.merge(dfs[0], dfs[1], how='left', left_on=['PERSON_ID', 'DISPLAY_FIRST_LAST'], right_on=['PLAYER_ID', 'PLAYER_NAME'])
                df = df.drop(['TimeFrame'], axis=1)
                print(player_id)
                return df
            except:
                continue
        print(player_id, "\n proxies failed; retrieving new proxies and attempting request again")
        proxies = get_quick_proxies()
            

common_player_info_dfs = []
for player_id in player_ids:
    common_player_info_df = dask.delayed(get_common_player_info)(int(player_id), proxies)
    common_player_info_dfs.append(common_player_info_df)

common_player_info_dfs = dask.persist(*common_player_info_dfs)
common_player_info_dfs = dask.compute(*common_player_info_dfs)
common_player_info_df = pd.concat(common_player_info_dfs)

common_player_info_df.head()

TypeError: cannot pickle '_hashlib.HASH' object

In [9]:
c.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


In [17]:
get_common_player_info(int(player_ids[0]), proxies)

76001 
 proxies failed; retrieving new proxies and attempting request again
--2021-03-18 14:02:29--  https://api.proxyscrape.com/v2/?request=getproxies&protocol=http&timeout=1000&country=all&ssl=all&anonymity=all&simplified=true
Resolving api.proxyscrape.com (api.proxyscrape.com)... 151.139.128.11
Connecting to api.proxyscrape.com (api.proxyscrape.com)|151.139.128.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3676 (3.6K) [text/plain]
Saving to: ‘http_proxies.txt’

http_proxies.txt    100%[===================>]   3.59K  --.-KB/s    in 0s      

2021-03-18 14:02:30 (9.93 MB/s) - ‘http_proxies.txt’ saved [3676/3676]

Original number of proxies:  185


KeyboardInterrupt: 

In [67]:
pd.concat([Player_Attributes, Player_Attributes])

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,PLAYER_ID,PLAYER_NAME,PTS,AST,REB,ALL_STAR_APPEARANCES
0,76001,Alaa,Abdelnaby,Alaa Abdelnaby,"Abdelnaby, Alaa",A. Abdelnaby,alaa-abdelnaby,1968-06-24T00:00:00,Duke,USA,...,Y,1990,1,25,76001,Alaa Abdelnaby,5.7,0.3,3.3,0
0,76001,Alaa,Abdelnaby,Alaa Abdelnaby,"Abdelnaby, Alaa",A. Abdelnaby,alaa-abdelnaby,1968-06-24T00:00:00,Duke,USA,...,Y,1990,1,25,76001,Alaa Abdelnaby,5.7,0.3,3.3,0


In [70]:
Player_Attributes.info

<bound method DataFrame.info of    PERSON_ID FIRST_NAME  LAST_NAME DISPLAY_FIRST_LAST  \
0      76001       Alaa  Abdelnaby     Alaa Abdelnaby   

  DISPLAY_LAST_COMMA_FIRST DISPLAY_FI_LAST     PLAYER_SLUG  \
0          Abdelnaby, Alaa    A. Abdelnaby  alaa-abdelnaby   

             BIRTHDATE SCHOOL COUNTRY  ... GAMES_PLAYED_FLAG DRAFT_YEAR  \
0  1968-06-24T00:00:00   Duke     USA  ...                 Y       1990   

  DRAFT_ROUND  DRAFT_NUMBER PLAYER_ID     PLAYER_NAME  PTS  AST  REB  \
0           1            25     76001  Alaa Abdelnaby  5.7  0.3  3.3   

  ALL_STAR_APPEARANCES  
0                    0  

[1 rows x 38 columns]>

In [ ]:
Player_Attributes